In [1]:
import pandas as pd

# Model definition

## XGB params:
- `max_depth`: 3
- `learning_rate`: 0.1
- `objective`: 'binary:logistic'
- `eval_metric`: 'logloss'
- `min_child_weight`: 1
- `subsample`: 0.8
- `colsample_bytree`: 0.8
- `min_child_weight`: 1
- `num_boost_round`: 100
- `early_stopping_rounds`: 10
- `seed`: 42

# Model Evaluation

In [6]:
pd.read_csv('../data/evaluations/eval.csv')

,accuracy,precision,recall,f1,auc
0,0.83,0.81,0.73,0.77,0.81


The model has been evaluated with all the common metrics for binary classification:
- accuracy: 0.83
- precision: 0.81
- recall: 0.73
- f1: 0.77
- ROC Curve and AUC: 0.81

All metrics are near 0.8 and indicate a good model.

The lowest metric is recall (0.73): this means the model is not that good at identifying positive cases, if the problem where such
that the importance of detecting positives was higher than the importance of detecting negative values it would be a good idea to change the model.
Maybe use un ensemble to better this metric

# Feature Importance

In [8]:
pd.read_csv('../data/feature_importances/feature_importance.csv').sort_values('importance', ascending=False)

,feature,importance
14,Title_Mr,0.444687
7,Sex_female,0.211257
0,Pclass,0.080986
4,FamilySize,0.053366
16,Title_Other,0.053197
11,Embarked_S,0.048922
12,Title_Master,0.044735
1,SibSp,0.036617
3,AgeBand,0.015754
6,FareBand,0.005737


As the objective function is not easily related to the features we choose to use "gain" as the type of feature importance.

This way we can see which features contribute the most in improving the performance of the model. (using the average gain of splits that use the feature)

The most important features are:
- `Title_Mr`: Make sense as "Women and children first" leaves adult men with less chance of survival
- `Sex_female`: Same as before
- `PClass`: Rich people survived more

# Putting the model in production

An option to put the cli in production is making an AWS Lambda + Gateway Api endpoint that triggers it.

- **Scalability**: AWS Lambda can scale to handle large numbers of requests automatically
- **Cost**: AWS Lambda and API Gateway are both "pay-as-you-go" services, which means that you only pay for the resources you use. This can be more cost-effective than running a traditional server that is always running.
- **Integration with other services**: AWS Lambda and API Gateway are easy to connect to other AWS services if there's need to interact with other resources
- **Security**: AWS Lambda and API Gateway are both designed to be secure by default, with features like built-in encryption, access control, and logging.

## Making the AWS Lambda code

The lambda receives the na,e of the command in `event.command` and the arguments necessary in `event.arguments`.

The cli package needs to be modified to save the outputs in the desired S3 bucket (usually a url received in the arguments)

````
import json
import logging
from ml_cli import main as ml_cli_main  # Import the ml_cli main function
import traceback

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    logger.info(f'Received event: {event}')

    # Get the command and arguments from the event
    command = event['command']
    arguments = event['arguments']

    try:
        # Call the ml_cli main function with the given command and arguments
        result = ml_cli_main([command] + arguments)
        logger.info(f'result: {result}')

        # Return the result as a JSON response
        return {
            'statusCode': 200,
            'body': json.dumps(result)
        }
    except Exception as e:
        # If an exception occurs, log the error and return a JSON response with the error message
        logger.error(f'Error: {traceback.format_exc()}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

````


## Cloudformation template

This Cloudformation template creates an AWS Lambda function from `ml_cli.zip` in the S3 bucket `ml-cli-bucket`  and connects it to an API Gateway:

The API Gateway has 1 endpoint for each cli command

```
Resources:
  # Define an IAM role for the Lambda function
  LambdaExecutionRole:
    Type: "AWS::IAM::Role"
    Properties:
      AssumeRolePolicyDocument:
        Version: "2012-10-17"
        Statement:
          - Effect: "Allow"
            Principal:
              Service: "lambda.amazonaws.com"
            Action:
              - "sts:AssumeRole"
      Path: "/"
      Policies:
        - PolicyName: "LambdaExecutionPolicy"
          PolicyDocument:
            Version: "2012-10-17"
            Statement:
              - Effect: "Allow"
                Action:
                  - "logs:CreateLogGroup"
                  - "logs:CreateLogStream"
                  - "logs:PutLogEvents"
                Resource: "arn:aws:logs:*:*:*"
              - Effect: "Allow"
                Action:
                  - "s3:GetObject"
                  - "s3:PutObject"
                Resource: "arn:aws:s3:::*"

  # Define the Lambda function
  MLCLIHandler:
    Type: "AWS::Lambda::Function"
    Properties:
      FunctionName: "ml_cli_lambda"
      Handler: "ml_cli.handler"
      Role: !GetAtt LambdaExecutionRole.Arn
      Runtime: "python3.9"
      Code:
        S3Bucket: "ml-cli-bucket"
        S3Key: "ml_cli.zip"
      Timeout: 30
      MemorySize: 256

  # Define the API Gateway
  MLCLIApi:
    Type: "AWS::ApiGateway::RestApi"
    Properties:
      Name: "ml_cli_api"
  MLCLIApiDeployment:
    Type: AWS::ApiGateway::Deployment
    Properties:
      RestApiId: !Ref MLCLIApi
      StageName: Prod
      
  # API Gateway resource and method for the train command
  TrainApiResource:
    Type: AWS::ApiGateway::Resource
    Properties:
      RestApiId: !Ref MLCLIApi
      ParentId: !GetAtt MLCLIApi.RootResourceId
      PathPart: train
  TrainApiMethod:
    Type: AWS::ApiGateway::Method
    Properties:
      RestApiId: !Ref MLCLIApi
      ResourceId: !Ref TrainApiResource
      HttpMethod: POST
      Integration:
        IntegrationHttpMethod: POST
        Type: AWS_PROXY
        Uri: !Sub "arn:aws:apigateway:${AWS::Region}:lambda:path/2015-03-31/functions/${MLCLIHandler.Arn}/invocations"
        PassthroughBehavior: WHEN_NO_MATCH
        RequestTemplates:
          application/json: |
            {
              "command": "train",
              "arguments": [
                "--data-file=$util.escapeJavaScript($input.path('$.data-file'))",
                "--model-file=$util.escapeJavaScript($input.path('$.model-file'))"
              ]
            }
      MethodResponses:
        - StatusCode: 200

  # API Gateway resource and method for the evaluate command
  EvaluateApiResource:
    Type: AWS::ApiGateway::Resource
    Properties:
      RestApiId: !Ref MLCLIApi
      ParentId: !GetAtt MLCLIApi.RootResourceId
      PathPart: evaluate
  EvaluateApiMethod:
    Type: AWS::ApiGateway::Method
    Properties:
      RestApiId: !Ref MLCLIApi
      ResourceId: !Ref EvaluateApiResource
      HttpMethod: POST
      Integration:
        IntegrationHttpMethod: POST
        Type: AWS_PROXY
        Uri: !Sub "arn:aws:apigateway:${AWS::Region}:lambda:path/2015-03-31/functions/${MLCLIHandler.Arn}/invocations"
        PassthroughBehavior: WHEN_NO_MATCH
        RequestTemplates:
          application/json: |
            {
              "command": "evaluate",
              "arguments": [
                "--input-file=$util.escapeJavaScript($input.path('$.input-file'))",
                "--model-file=$util.escapeJavaScript($input.path('$.model-file'))"
                "--output-file=$util.escapeJavaScript($input.path('$.output-file'))"
              ]
            }
      MethodResponses:
        - StatusCode: 200

  # API Gateway resource and method for the predict command
  PredictApiResource:
    Type: AWS::ApiGateway::Resource
    Properties:
      RestApiId: !Ref MLCLIApi
      ParentId: !GetAtt MLCLIApi.RootResourceId
      PathPart: predict
  PredictApiMethod:
    Type: AWS::ApiGateway::Method
    Properties:
      RestApiId: !Ref MLCLIApi
      ResourceId: !Ref PredictApiResource
      HttpMethod: POST
      Integration:
        IntegrationHttpMethod: POST
        Type: AWS_PROXY
        Uri: !Sub "arn:aws:apigateway:${AWS::Region}:lambda:path/2015-03-31/functions/${MLCLIHandler.Arn}/invocations"
        PassthroughBehavior: WHEN_NO_MATCH
        RequestTemplates:
          application/json: |
            {
              "command": "predict",
              "arguments": [
                "--input-file=$util.escapeJavaScript($input.path('$.input-file'))",
                "--model-file=$util.escapeJavaScript($input.path('$.model-file'))"
                "--output-file=$util.escapeJavaScript($input.path('$.output-file'))"
              ]
            }
      MethodResponses:
        - StatusCode: 200

  # API Gateway resource and method for the getFeatureImportances command
  GetFeatureImportancesApiResource:
    Type: AWS::ApiGateway::Resource
    Properties:
      RestApiId: !Ref MLCLIApi
      ParentId: !GetAtt MLCLIApi.RootResourceId
      PathPart: get-feature-importances
  GetFeatureImportancesApiMethod:
    Type: AWS::ApiGateway::Method
    Properties:
      RestApiId: !Ref MLCLIApi
      ResourceId: !Ref GetFeatureImportancesApiResource
      HttpMethod: POST
      Integration:
        IntegrationHttpMethod: POST
        Type: AWS_PROXY
        Uri: !Sub "arn:aws:apigateway:${AWS::Region}:lambda:path/2015-03-31/functions/${MLCLIHandler.Arn}/invocations"
        PassthroughBehavior: WHEN_NO_MATCH
        RequestTemplates:
          application/json: |
            {
              "command": "get-feature-importances",
              "arguments": [
                "--model-file=$util.escapeJavaScript($input.path('$.model-file'))"
                "--output-file=$util.escapeJavaScript($input.path('$.output-file'))"
              ]
            }
      MethodResponses:
        - StatusCode: 200
```


## Automating updates to Lambda code

- The lambda code should go in a github repo
- Create a Code pipeline with the github repository as the source
- Add Codebuild project to the pipeline that builds the lambda code each time the main branch of the repo changes
- Add Codedeploy step that deploys the new lambda code to the lambda

Example Cloudformation template:
```
Resources:
  MyLambdaFunction:
    Type: AWS::Lambda::Function
    Properties:
      Code:
        S3Bucket: !Ref LambdaCodeBucket
        S3Key: !Ref LambdaCodeKey
      Handler: lambda_function.lambda_handler
      Role: !GetAtt MyLambdaRole.Arn
      Runtime: python3.9

  MyLambdaRole:
    Type: AWS::IAM::Role
    Properties:
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
        - Effect: Allow
          Principal:
            Service: lambda.amazonaws.com
          Action:
          - sts:AssumeRole
      ManagedPolicyArns:
      - arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole

  MyPipeline:
    Type: AWS::CodePipeline::Pipeline
    Properties:
      Name: my-pipeline
      RoleArn: !GetAtt MyPipelineRole.Arn
      ArtifactStore:
        Type: S3
        Location: !Ref PipelineBucket
      Stages:
      - Name: Source
        Actions:
        - Name: SourceAction
          ActionTypeId:
            Category: Source
            Owner: ThirdParty
            Provider: GitHub
            Version: 1
          OutputArtifacts:
          - Name: SourceCode
          Configuration:
            Owner: !Ref GitHubOwner
            Repo: !Ref GitHubRepo
            Branch: !Ref GitHubBranch
            OAuthToken: !Ref GitHubToken
        - Name: BuildAction
          ActionTypeId:
            Category: Build
            Owner: AWS
            Provider: CodeBuild
            Version: 1
          InputArtifacts:
          - Name: SourceCode
          OutputArtifacts:
          - Name: BuildOutput
          Configuration:
            ProjectName: !Ref BuildProject
      - Name: Deploy
        Actions:
        - Name: DeployAction
          ActionTypeId:
            Category: Deploy
            Owner: AWS
            Provider: CodeDeployToLambda
            Version: 1
          InputArtifacts:
          - Name: BuildOutput
          Configuration:
            FunctionName: !GetAtt MyLambdaFunction.Arn
            DeploymentGroupName: !Ref CodeDeployDeploymentGroup
            RoleArn: !GetAtt MyCodeDeployRole.Arn

  MyPipelineRole:
    Type: AWS::IAM::Role
    Properties:
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
        - Effect: Allow
          Principal:
            Service: codepipeline.amazonaws.com
          Action:
          - sts:AssumeRole
      Policies:
      - PolicyName: my-pipeline-policy
        PolicyDocument:
          Version: '2012-10-17'
          Statement:
          - Effect: Allow
            Action:
            - s3:*
            Resource: !Sub arn:aws:s3:::${PipelineBucket}/*
          - Effect: Allow
            Action:
            - s3:
```
